In [4]:
#MODEL TREE
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR    = "cv_folds"
PLAYER_FEATS = ["PTS", "REB", "AST"]
OPP_FEATS    = ["opp_E_OFF_RATING", "opp_E_DEF_RATING", "opp_E_NET_RATING", "opp_REB_PCT"]
TARGET_COLS  = ["PTS", "REB", "AST"]
N_JOBS       = 4

# ─── Hyperparameters to try ─────────────────────────────────────────────────
hyperparams = [
    {"max_depth": None},
    {"max_depth": 3},
    {"max_depth": 5},
    {"max_depth": 10},
    {"max_depth": 20},
]

# ─── Sequence builders ─────────────────────────────────────────────────────
def build_train_sequences(df):
    X_seq, X_opp, Y = [], [], []
    for pid, gp in df.groupby("PLAYER_ID"):
        gp = gp.sort_values("GAME_DATE")
        p = gp[PLAYER_FEATS].values
        o = gp[OPP_FEATS].values
        for i in range(8, len(gp)):
            X_seq.append(p[i-8:i])
            X_opp.append(o[i])
            Y.append(p[i])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

def build_test_sequences(train_df, test_df):
    X_seq, X_opp, Y = [], [], []
    for row in test_df.sort_values("GAME_DATE").itertuples():
        pid = row.PLAYER_ID
        hist = train_df[train_df.PLAYER_ID == pid].sort_values("GAME_DATE")
        if len(hist) < 8:
            continue
        p = hist[PLAYER_FEATS].values
        X_seq.append(p[-8:])
        o_vec = [getattr(row, f) for f in OPP_FEATS]
        X_opp.append(o_vec)
        Y.append([row.PTS, row.REB, row.AST])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

# ─── Single-fold evaluation ─────────────────────────────────────────────────
def eval_tree_fold(fold_idx, hp):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])

    # build sequences
    X_tr_seq, X_tr_opp, y_tr = build_train_sequences(tr)
    X_te_seq, X_te_opp, y_te = build_test_sequences(tr, te)

    # flatten sequence and combine with opponent stats
    ns, ts, nf = X_tr_seq.shape
    X_tr_flat = X_tr_seq.reshape(ns, ts * nf)
    X_te_flat = X_te_seq.reshape(X_te_seq.shape[0], ts * nf)
    X_tr = np.hstack([X_tr_flat, X_tr_opp])
    X_te = np.hstack([X_te_flat, X_te_opp])

    # instantiate multi-output decision tree
    base = DecisionTreeRegressor(max_depth=hp["max_depth"], random_state=42)
    model = MultiOutputRegressor(base)
    model.fit(X_tr, y_tr)

    # predict & compute metrics
    preds = model.predict(X_te)
    metrics = {}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_te[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run evaluation ─────────────────────────────────────────────────────────
all_results = []
for hp in hyperparams:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(eval_tree_fold)(fold, hp) for fold in range(1, 11)
    )
    df = pd.DataFrame(fold_metrics)
    df["fold"]      = np.arange(1, len(df) + 1)
    df["max_depth"] = hp["max_depth"]
    all_results.append(df)

# concatenate and reorder
final_df = pd.concat(all_results, ignore_index=True)
cols = ["max_depth", "fold"] + [c for c in final_df.columns if c not in ("max_depth", "fold")]
final_df = final_df[cols]

# display results
print(final_df)


   max_depth  fold   PTS_rmse   PTS_mae    PTS_r2  REB_rmse   REB_mae  \
0       None     1  10.387278  8.139671 -0.416988  3.878586  2.977700   
1       None     2  10.382246  8.234222 -0.412447  3.967876  3.021739   
2       None     3   9.369472  7.285000 -0.220843  3.957777  2.974000   
3       None     4   9.787389  7.706035 -0.201464  4.088254  3.099286   
4       None     5  10.195477  7.929233 -0.340334  3.854496  2.892857   
5       None     6  10.403731  8.134653 -0.328390  4.125666  3.121452   
6       None     7  10.154045  8.023148 -0.359429  3.905955  2.947222   
7       None     8  10.038618  7.884967 -0.221365  3.968998  3.043137   
8       None     9  10.111741  7.968919 -0.458701  4.020387  3.068919   
9       None    10  10.289171  7.963739 -0.312835  4.227228  3.210314   
10         3     1  10.111578  7.871539 -0.342767  3.620034  2.755045   
11         3     2   7.596449  5.989318  0.243845  2.844369  2.204461   
12         3     3   7.292787  5.875600  0.260367  

In [3]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR    = "cv_folds"
PLAYER_FEATS = ["PTS", "REB", "AST"]
OPP_FEATS    = ["opp_E_OFF_RATING", "opp_E_DEF_RATING", "opp_E_NET_RATING", "opp_REB_PCT"]
TARGET_COLS  = ["PTS", "REB", "AST"]
N_JOBS       = 4

# ─── Hyperparameters to try ─────────────────────────────────────────────────
hyperparams = [
    {"n_estimators": 100, "max_depth": None},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": None},
    {"n_estimators": 200, "max_depth": 10},
]

# ─── Sequence builders ─────────────────────────────────────────────────────
def build_train_sequences(df):
    X_seq, X_opp, Y = [], [], []
    for pid, gp in df.groupby("PLAYER_ID"):
        gp = gp.sort_values("GAME_DATE")
        p = gp[PLAYER_FEATS].values
        o = gp[OPP_FEATS].values
        for i in range(8, len(gp)):
            X_seq.append(p[i-8:i])
            X_opp.append(o[i])
            Y.append(p[i])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

def build_test_sequences(train_df, test_df):
    X_seq, X_opp, Y = [], [], []
    for row in test_df.sort_values("GAME_DATE").itertuples():
        pid = row.PLAYER_ID
        hist = train_df[train_df.PLAYER_ID == pid].sort_values("GAME_DATE")
        if len(hist) < 8:
            continue
        p = hist[PLAYER_FEATS].values
        X_seq.append(p[-8:])
        o_vec = [getattr(row, f) for f in OPP_FEATS]
        X_opp.append(o_vec)
        Y.append([row.PTS, row.REB, row.AST])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

# ─── Single-fold evaluation ─────────────────────────────────────────────────
def eval_rf_fold(fold_idx, hp):
    # load data
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])

    # build sequences
    X_tr_seq, X_tr_opp, y_tr = build_train_sequences(tr)
    X_te_seq, X_te_opp, y_te = build_test_sequences(tr, te)

    # flatten sequence and combine with opponent stats
    ns, ts, nf = X_tr_seq.shape
    X_tr_flat = X_tr_seq.reshape(ns, ts * nf)
    X_te_flat = X_te_seq.reshape(X_te_seq.shape[0], ts * nf)
    X_tr = np.hstack([X_tr_flat, X_tr_opp])
    X_te = np.hstack([X_te_flat, X_te_opp])

    # instantiate multi-output random forest
    base = RandomForestRegressor(
        n_estimators=hp["n_estimators"],
        max_depth=hp["max_depth"],
        random_state=42,
        n_jobs=1
    )
    model = MultiOutputRegressor(base)
    model.fit(X_tr, y_tr)

    # predict & compute metrics
    preds = model.predict(X_te)
    metrics = {}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_te[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run evaluation ─────────────────────────────────────────────────────────
all_results = []
for hp in hyperparams:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(eval_rf_fold)(fold, hp) for fold in range(1, 11)
    )
    df = pd.DataFrame(fold_metrics)
    df["fold"]         = np.arange(1, len(df) + 1)
    df["n_estimators"] = hp["n_estimators"]
    df["max_depth"]    = hp["max_depth"]
    all_results.append(df)

# concatenate all runs
final_df = pd.concat(all_results, ignore_index=True)

# reorder columns
cols = ["n_estimators", "max_depth", "fold"] + [c for c in final_df.columns if c not in ("n_estimators", "max_depth", "fold")]
final_df = final_df[cols]

# display results
print(final_df)


/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


    n_estimators max_depth  fold  PTS_rmse   PTS_mae    PTS_r2  REB_rmse  \
0            100      None     1  7.870420  6.312254  0.186498  2.933120   
1            100      None     2  6.903463  5.442938  0.375512  2.659916   
2            100      None     3  6.650606  5.347510  0.384891  2.744042   
3            100      None     4  6.994661  5.509812  0.386365  2.838868   
4            100      None     5  6.874259  5.430456  0.390674  2.790628   
5            100      None     6  6.946533  5.451109  0.407779  2.829126   
6            100      None     7  7.008350  5.557685  0.352396  2.864051   
7            100      None     8  7.240319  5.548039  0.364650  2.801048   
8            100      None     9  6.733415  5.312230  0.353178  2.721099   
9            100      None    10  7.349168  5.809468  0.330230  2.976763   
10           100        10     1  7.870420  6.312254  0.186498  2.933120   
11           100        10     2  6.895461  5.425281  0.376959  2.659999   
12          

In [5]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Concatenate, LayerNormalization, 
    MultiHeadAttention, Dropout, Add, GlobalAveragePooling1D
)

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR    = "cv_folds"
PLAYER_FEATS = ["PTS", "REB", "AST"]
OPP_FEATS    = ["opp_E_OFF_RATING", "opp_E_DEF_RATING", "opp_E_NET_RATING", "opp_REB_PCT"]
TARGET_COLS  = ["PTS", "REB", "AST"]
N_JOBS       = 4
BATCH_SIZE   = 32

# ─── Hyperparameters to try ─────────────────────────────────────────────────
hyperparams = [
    {"head_size": 64,  "num_heads": 2, "ff_dim": 64,  "dropout": 0.1, "epochs": 10},
    {"head_size": 64,  "num_heads": 4, "ff_dim": 64,  "dropout": 0.1, "epochs": 10},
    {"head_size": 128, "num_heads": 4, "ff_dim": 128, "dropout": 0.1, "epochs": 20},
    {"head_size": 256, "num_heads": 4, "ff_dim": 256, "dropout": 0.1, "epochs": 20},
]

# ─── Transformer Encoder Block ──────────────────────────────────────────────
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout):
    # pre‐norm
    x = LayerNormalization(epsilon=1e-6)(inputs)
    # self-attention
    x = MultiHeadAttention(
        key_dim=head_size, 
        num_heads=num_heads, 
        dropout=dropout
    )(x, x)
    x = Dropout(dropout)(x)
    res = Add()([x, inputs])
    
    # feed-forward
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    x = Dropout(dropout)(x)
    return Add()([x, res])

# ─── Build dual‐input Transformer model ─────────────────────────────────────
def make_dual_transformer(timesteps, n_feats, n_opp, n_outputs, hp):
    # sequence input
    seq_in = Input(shape=(timesteps, n_feats), name="player_seq")
    # one transformer block
    x = transformer_encoder(
        seq_in, 
        head_size=hp["head_size"], 
        num_heads=hp["num_heads"], 
        ff_dim=hp["ff_dim"], 
        dropout=hp["dropout"]
    )
    # pool over time
    x = GlobalAveragePooling1D()(x)

    # opponent stats input
    opp_in = Input(shape=(n_opp,), name="opp_stats")
    # combine
    m = Concatenate()([x, opp_in])

    # dense head → outputs
    h = Dense(16, activation="relu")(m)
    out = Dense(n_outputs)(h)

    model = Model([seq_in, opp_in], out)
    model.compile(optimizer="adam", loss="mse")
    return model

# ─── Sequence builders ─────────────────────────────────────────────────────
def build_train_sequences(df):
    X_seq, X_opp, Y = [], [], []
    for pid, gp in df.groupby("PLAYER_ID"):
        gp = gp.sort_values("GAME_DATE")
        p = gp[PLAYER_FEATS].values
        o = gp[OPP_FEATS].values
        for i in range(8, len(gp)):
            X_seq.append(p[i-8:i])
            X_opp.append(o[i])
            Y.append(p[i])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

def build_test_sequences(train_df, test_df):
    X_seq, X_opp, Y = [], [], []
    for row in test_df.sort_values("GAME_DATE").itertuples():
        pid = row.PLAYER_ID
        hist = train_df[train_df.PLAYER_ID == pid].sort_values("GAME_DATE")
        if len(hist) < 8:
            continue
        p = hist[PLAYER_FEATS].values
        X_seq.append(p[-8:])
        o_vec = [getattr(row, f) for f in OPP_FEATS]
        X_opp.append(o_vec)
        Y.append([row.PTS, row.REB, row.AST])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

# ─── Single‐fold evaluation ─────────────────────────────────────────────────
def eval_transformer_fold(fold_idx, hp):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])

    # build sequences
    X_tr_seq, X_tr_opp, y_tr = build_train_sequences(tr)
    X_te_seq, X_te_opp, y_te = build_test_sequences(tr, te)

    # scale sequence features
    ns, ts, nf = X_tr_seq.shape
    seq_scaler = StandardScaler().fit(X_tr_seq.reshape(-1, nf))
    X_tr_seq = seq_scaler.transform(X_tr_seq.reshape(-1, nf)).reshape(ns, ts, nf)
    X_te_seq = seq_scaler.transform(X_te_seq.reshape(-1, nf)).reshape(-1, ts, nf)

    # scale opponent features
    opp_scaler = StandardScaler().fit(X_tr_opp)
    X_tr_opp = opp_scaler.transform(X_tr_opp)
    X_te_opp = opp_scaler.transform(X_te_opp)

    # build & train model
    model = make_dual_transformer(ts, nf, len(OPP_FEATS), len(TARGET_COLS), hp)
    model.fit(
        [X_tr_seq, X_tr_opp], y_tr,
        epochs=hp["epochs"],
        batch_size=BATCH_SIZE,
        verbose=0
    )

    # predict & metrics
    preds = model.predict([X_te_seq, X_te_opp], verbose=0)
    metrics = {}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_te[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run evaluation ─────────────────────────────────────────────────────────
all_results = []
for hp in hyperparams:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(eval_transformer_fold)(fold, hp) for fold in range(1, 11)
    )
    df = pd.DataFrame(fold_metrics)
    df["fold"]        = np.arange(1, len(df) + 1)
    df["head_size"]   = hp["head_size"]
    df["num_heads"]   = hp["num_heads"]
    df["ff_dim"]      = hp["ff_dim"]
    df["dropout"]     = hp["dropout"]
    df["epochs"]      = hp["epochs"]
    all_results.append(df)

# concatenate & reorder
final_df = pd.concat(all_results, ignore_index=True)
cols = ["head_size", "num_heads", "ff_dim", "dropout", "epochs", "fold"] + [
    c for c in final_df.columns if c not in ("head_size","num_heads","ff_dim","dropout","epochs","fold")
]
final_df = final_df[cols]

# display
print(final_df)


2025-05-05 00:10:42.403200: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-05 00:10:42.403227: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-05 00:10:42.403230: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-05 00:10:42.403245: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-05 00:10:42.403253: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-05-05 00:10:42.421287: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-05 00:10:42.421308: I metal_plugin/src/device/metal_device.cc:296] syste

    head_size  num_heads  ff_dim  dropout  epochs  fold   PTS_rmse    PTS_mae  \
0          64          2      64      0.1      10     1  16.733638  14.300495   
1          64          2      64      0.1      10     2   7.381012   5.860282   
2          64          2      64      0.1      10     3   7.249900   5.861605   
3          64          2      64      0.1      10     4   7.607123   5.967729   
4          64          2      64      0.1      10     5   6.826942   5.387874   
5          64          2      64      0.1      10     6   7.158997   5.641006   
6          64          2      64      0.1      10     7   7.155447   5.768554   
7          64          2      64      0.1      10     8   7.277671   5.555387   
8          64          2      64      0.1      10     9   7.134048   5.644553   
9          64          2      64      0.1      10    10   7.307081   5.746543   
10         64          4      64      0.1      10     1  16.964034  14.581117   
11         64          4    